<a href="https://colab.research.google.com/github/sumankmaiti/All-in-one/blob/main/vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [ ]:

"""We'll also set the random seeds."""

### Defining the Model 

class VGG(nn.Module):
    def __init__(self, features, output_dim):
        super().__init__()

        self.features = features

        self.avgpool = nn.AdaptiveAvgPool2d(7)

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        # x = F.softmax(x, dim=-1)

        return x, h


In [ ]:
""" Next up is calculating the `features` for each VGG configuration."""

vgg11_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

vgg13_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512,
                512, 'M']

vgg16_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512,
                'M', 512, 512, 512, 'M']

vgg19_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512,
                512, 512, 'M', 512, 512, 512, 512, 'M']


In [ ]:
def get_vgg_layers(config, batch_norm):

    layers = []
    in_channels = 3

    for c in config:
        assert c == 'M' or isinstance(c, int)
        if c == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, c, kernel_size=3, stride=1, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = c

    return nn.Sequential(*layers)

"""Now, let's get the `features` for the VGG11 architecture, with batch normalization."""

vgg19_layers = get_vgg_layers(vgg19_config, batch_norm=False)
vgg19_layers
"""We can print them out and ensure they are the same as the "A" configuration of the VGG configuration table."""

print(vgg19_layers)
with open('log_vgg19_imagenet_lr_change.txt', 'a') as f:
        f.write(str(vgg19_layers))
        f.write('\n')

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [ ]:
"""We can then pass these `features` to our base VGG module to get our VGG11 model."""

OUTPUT_DIM = 10

model = VGG(vgg19_layers, OUTPUT_DIM)

print(model)
with open('log_vgg19_imagenet_lr_change.txt', 'a') as f:
        f.write(str(model))
        f.write('\n')


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
pretrained_size = [224, 224]
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(pretrained_size, padding=10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=pretrained_means,
                                                std=pretrained_stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=pretrained_means,
                                                std=pretrained_stds)
                       ])

In [ ]:
# """...then create the validation split..."""
# ROOT = 'dataset'

# train_data = datasets.CIFAR10(ROOT,
#                               train=True,
#                               download=True,
#                               transform=train_transforms)

# test_data = datasets.CIFAR10(ROOT,
#                              train=False,
#                              download=True,
#                              transform=test_transforms)

# VALID_RATIO = 0.9

# n_train_examples = int(len(train_data) * VALID_RATIO)
# n_valid_examples = len(train_data) - n_train_examples

# train_data, valid_data = data.random_split(train_data,
#                                            [n_train_examples, n_valid_examples])


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataset/cifar-10-python.tar.gz to dataset
Files already downloaded and verified


In [ ]:
import torchvision
train_data = torchvision.datasets.ImageFolder("/content/drive/MyDrive/train_last_10/", transform= train_transforms)
VALID_RATIO = 0.8

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])


In [ ]:
"""...and ensure the validation data uses the test transforms."""

valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

"""Again, we'll print out the number of examples in each split to ensure they are correct."""

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
#print(f'Number of testing examples: {len(test_data)}')
with open('log_vgg19_imagenet_lr_change.txt', 'a') as f:
        f.write(f'Number of training examples: {len(train_data)}\n')
        f.write(f'Number of validation examples: {len(valid_data)}\n')
#        f.write(f'Number of testing examples: {len(test_data)}\n')
        f.write('\n') 

Number of training examples: 10400
Number of validation examples: 2600


In [ ]:
"""As the model is ~5x the size of the AlexNet model from the previous notebook we use a smaller batch size so it can fit on reasonably sized GPUs. This should be increased if we have access to GPUs with more memory in order to speed up training."""

BATCH_SIZE = 8

train_iterator = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

#test_iterator = data.DataLoader(test_data,
#                                batch_size=BATCH_SIZE)


In [ ]:

"""### Training the Model

We'll use the learning rate finder as used in previous notebooks. Generally when using a pre-trained model the learning rate used will be considerably lower.

First, we'll set up the optimizer with the initial learning rate that is much lower than we expect to use. Then we define the `device` to place our model on our GPU, if we have one. Next we define the `criterion` (loss function) and place the model and criterion on our device.
"""

START_LR = 0.0001

optimizer = optim.Adam(model.parameters(), lr=START_LR, betas=(0.9, 0.999), weight_decay = 0.0005)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()

model = nn.DataParallel(model)
model = model.to(device)

criterion = criterion.to(device)

In [ ]:
"""Now all of the set-up is done, the rest of the notebook is pretty standard from here on out.

We create a function to calculate accuracy...
"""

def calculate_accuracy(y_pred, y):
    # print(y_pred)
    # print(y.shape[0])
    top_pred = y_pred.argmax(1)
    # print(top_pred)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]   
    # print(correct)
    # print(acc)
    return acc


In [ ]:
"""...create a function that implements a training loop..."""

def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in iterator:

        x = x.to(device)
        y = y.to(device)
        # print(y)

        optimizer.zero_grad()

        y_pred, _ = model(x)
        # print(_)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    # print(epoch_loss, epoch_acc, len(iterator))

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:

"""...create a function that performs an evaluation loop..."""

def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            model.to(device)
            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
"""...and a helper function to tell us how long an epoch takes."""

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [ ]:
"""Finally, we train our model."""

EPOCHS = 100

best_valid_loss = float('inf')

for epoch in range(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'vgg19_imagenet_lr_change.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    with open('log_vgg19_imagenet_lr_change.txt', 'a') as f:
        f.write(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        f.write(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        f.write(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        f.write('\n')

Epoch: 01 | Epoch Time: 59m 47s
	Train Loss: 2.303 | Train Acc: 9.83%
	 Val. Loss: 2.303 |  Val. Acc: 10.58%
Epoch: 02 | Epoch Time: 14m 17s
	Train Loss: 2.303 | Train Acc: 9.48%
	 Val. Loss: 2.303 |  Val. Acc: 9.38%
Epoch: 03 | Epoch Time: 14m 5s
	Train Loss: 2.303 | Train Acc: 9.74%
	 Val. Loss: 2.303 |  Val. Acc: 9.50%
Epoch: 04 | Epoch Time: 13m 53s
	Train Loss: 2.303 | Train Acc: 9.82%
	 Val. Loss: 2.303 |  Val. Acc: 9.50%
Epoch: 05 | Epoch Time: 14m 12s
	Train Loss: 2.303 | Train Acc: 9.58%
	 Val. Loss: 2.303 |  Val. Acc: 9.50%


KeyboardInterrupt: ignored

In [ ]:
"""Our test accuracy is a little lower at ~92%, but is still higher than the ~75% achieved by the AlexNet model in the previous tutorial notebook."""

model.load_state_dict(torch.load('vgg19_imagenet_lr_change.pt'))

test_loss, test_acc = evaluate(model, valid_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
with open('log_vgg19_imagenet_lr_change.txt', 'a') as f:
        f.write(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
        f.write('\n')

In [ ]:
def get_predictions(model, iterator):

    model.eval()

    images = []
    labels = []
    probs = []

    with torch.no_grad():

        for (x, y) in iterator:

            x = x.to(device)

            y_pred, _ = model(x)

            y_prob = F.softmax(y_pred, dim=-1)

            images.append(x.cpu())
            labels.append(y.cpu())
            probs.append(y_prob.cpu())

    images = torch.cat(images, dim=0)
    labels = torch.cat(labels, dim=0)
    probs = torch.cat(probs, dim=0)

    return images, labels, probs

images, labels, probs = get_predictions(model, valid_iterator)

"""...then get the predicted labels for each image..."""

pred_labels = torch.argmax(probs, 1)

print("prdected lebels:",pred_labels)
print("original lebels:",labels)
with open('log_vgg19_imagenet_lr_change.txt', 'a') as f:
        f.write("prdected lebels:",pred_labels)
        f.write("original lebels:",labels)
        f.write('\n')



In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
print(output)
output.backward()

tensor([[-0.4448, -0.4471, -0.7386, -0.0355, -1.1138],
        [-0.7687, -1.1237,  0.7376, -0.8181,  0.7374],
        [ 1.1422, -0.3587, -0.7909, -0.4120, -1.1980]], requires_grad=True)
tensor([0, 2, 4])
tensor(1.7892, grad_fn=<NllLossBackward0>)
